# Spam Farm Analysis Demo

This notebook demonstrates the complete spam analysis pipeline.

In [ ]:
import sys
import os
sys.path.insert(0, 'src')

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from spam_analysis_pipeline import run_complete_spam_experiment

## 1. Load or Create Sample Graph

In [ ]:
# Create sample graph
G = nx.erdos_renyi_graph(1000, 0.01, seed=42)
print(f"Graph: {len(G)} nodes, {G.number_of_edges()} edges")

## 2. Configure Experiment

In [ ]:
experiment_config = {
    'spam_type': 'simple',
    'm': 500,  # supporting pages
    'external_links': 10,
    'beta': 0.85,
    'detection_methods': ['trustrank', 'spam_mass', 'structural'],
    'trusted_nodes': None  # Will auto-select
}

## 3. Run Complete Experiment

In [ ]:
results = run_complete_spam_experiment(
    G, 
    experiment_config,
    save_intermediate=True,
    output_dir='results/demo'
)

## 4. Analyze Results

In [ ]:
# Detection evaluation
if results.detection_evaluations:
    print("Detection Results:")
    for method, eval_data in results.detection_evaluations.items():
        if isinstance(eval_data, dict) and 'metrics' in eval_data:
            metrics = eval_data['metrics']
            print(f"\n{method}:")
            print(f"  Precision: {metrics.get('precision', 0):.4f}")
            print(f"  Recall: {metrics.get('recall', 0):.4f}")
            print(f"  F1-Score: {metrics.get('f1_score', 0):.4f}")

In [ ]:
# Effectiveness analysis
if results.effectiveness_analysis:
    if 'amplification' in results.effectiveness_analysis:
        amp_df = pd.DataFrame(results.effectiveness_analysis['amplification'])
        print("\nPageRank Amplification:")
        print(amp_df[['target', 'amplification', 'predicted_amplification']])
    
    if 'roi' in results.effectiveness_analysis:
        roi_df = pd.DataFrame(results.effectiveness_analysis['roi'])
        print("\nROI Analysis:")
        print(roi_df[['target', 'cost', 'benefit', 'roi']])

## 5. Visualizations

In [ ]:
from spam_effectiveness_analysis import visualize_spam_impact

plot_paths = visualize_spam_impact(
    results.G_original if hasattr(results, 'G_original') else G,
    results.G_with_spam,
    results.spam_metadata,
    'results/demo/plots'
)

print("Plots saved to:")
for name, path in plot_paths.items():
    print(f"  {name}: {path}")